# Introduction

We show in this Kernel how we can process the data to prepare it for easier processing. Let's check the data files.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Analysis preparation

## Load packages

In [ ]:
import pandas as pd

## Load the data

The datafiles are in TSV format. We will read the files using pandas, just include in the function call the `sep` (tab separator data).
We demonstrate first how to read and process the Annual data.

In [ ]:
data_df = pd.read_csv("/kaggle/input/passengers-air-transport-in-europe/ttr00012.tsv", sep='\t')

Let's glimpse the data columns.

In [ ]:
print(list(data_df.columns))

The first column is a composed one, containing 5 different information (unit, sex, age, isced-11 education attainment levels classification and geography). The next columns are the year value, from last (2020) to first (1983).

# Data pre-processing

We start by defining two working lists.

In [ ]:
pivot_data_col = data_df.columns[0]
time_columns = data_df.columns[1:]

Then, we split from `pivot_data_col` the 3 separate fields:
* unit;
* tra_meas;
* tra_cov;
* schedule;
* geography (countries and Euro country groups).

In [ ]:
data_df['unit'] = data_df[pivot_data_col].apply(lambda x: x.split(",")[0])
data_df['tra_meas']     = data_df[pivot_data_col].apply(lambda x: x.split(",")[1])
data_df['tra_cov'] = data_df[pivot_data_col].apply(lambda x: x.split(",")[2])
data_df['schedule'] = data_df[pivot_data_col].apply(lambda x: x.split(",")[3])
data_df['geography'] = data_df[pivot_data_col].apply(lambda x: x.split(",")[4])

We select now only the new columns resulted from splitting the `pivot_data_col` and the time columns.

In [ ]:
selected_columns = list(['unit', 'tra_meas', 'tra_cov', 'schedule', 'geography']) +  list(time_columns)
data_df = data_df[selected_columns]

Next, we pivot the time columns using `melt` operation in pandas.  
We also make sure we transform `date` to be an integer (here is a year data).  
We set `value` to be a float, after we replace ": " (for N/A) with `NAN`.

In [ ]:
data_tr_df = data_df.melt(id_vars=['unit', 'tra_meas', 'tra_cov', 'schedule', 'geography'], 
        var_name="date", 
        value_name="value")
data_tr_df['date'] = data_tr_df['date'].apply(lambda x: int(x))
data_tr_df['value'] = data_tr_df['value'].apply(lambda x: str(x).replace("bp", ""))
data_tr_df['value'] = data_tr_df['value'].apply(lambda x: str(x).replace("b", ""))
data_tr_df['value'] = data_tr_df['value'].apply(lambda x: str(x).replace("u", ""))
data_tr_df['value'] = data_tr_df['value'].apply(lambda x: str(x).replace("c", ""))
data_tr_df['value'] = data_tr_df['value'].apply(lambda x: str(x).replace("d", ""))
data_tr_df['value'] = data_tr_df['value'].apply(lambda x: str(x).replace(": ", "NAN"))
data_tr_df['value'] = data_tr_df['value'].apply(lambda x: float(x))

Let's inspect the result.

In [ ]:
print(f"Transformed data shape: {data_tr_df.shape} (rows/columns)")
data_tr_df.head()

In [ ]:
data_tr_df.tail()

# A very preliminary exploratory data analysis

This would be a very short exploratory data analysis. The role of this Kernel is just to show how we can prepare the annual data for analysis and we already did this.

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(data_tr_df)

# Export data in csv format

In [ ]:
data_tr_df.to_csv("passengers_air_transport.csv", index=False)